<a href="https://colab.research.google.com/github/bwowby/DS/blob/master/ir-basic/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word Embedding
희소(sparse) 벡터인 원핫벡터와는 달리 단어를 밀집(dense) 벡터로 표현하는 방법.   
LSA,Word2Vec,FastText,Glove 등의 방법론이 있다.


### Word2Vec

영어/한국어 Word2Vec 실습   
https://wikidocs.net/50739

#### 필요한 모듈 import

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

#### 데이터 불러오기 및 전처리

In [6]:
# xml로 되어있는 데이터 다운로드 (아마 테드 강연 대본인듯)
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7efec02eae10>)

In [13]:
# xml 데이터 파일은 자연어를 얻기 위해 전처리가 필요함 -> 태그들 제거하고 데이터만 가져와야한다
# 내용 중에는 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 제거
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로 부터 <content> 태그 사이의 내용만 가져옴
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 괄호가 붙은 Audio, Laughter 등의 배경음 제거
content_text = re.sub(r'\([^)]*\)','',parse_text)

# 입력 corpus에 대해 nltk를 이용해 문장 토큰화 수행
sent_text = sent_tokenize(content_text)

# 각 문장에서 구두점 제거 및 대문자->소문자 변환
normalized_text=[]
for string in sent_text:
  tokens = re.sub(r"[^a-z0-9]+"," ",string.lower())
  normalized_text.append(tokens)

# 각 문장에 대해 nltk를 이용해 단어 토큰화 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

In [14]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [20]:
# 샘플 2개만 출력
for line in result[:2] :
  print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


#### Word2Vec 훈련시키기

In [21]:
from gensim.models import Word2Vec

'''
하이퍼 파라미터 값
size = 워드벡터의 특징 값, 차원
window = 컨텍스트 윈도우 크기
min_count = 단어 최소 빈도수 제한(빈도가 적은 단어는 학습 ㄴㄴ)
workers = 학습위한 프로세스 수 
sg = 0 :CBOW, 1은 Skip-gram ???? 
'''
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [22]:
# man 과 가장 유사한 단어 출력해보기 -> man,guy,boy,lady,girl.. 0o0
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8388179540634155), ('guy', 0.8041568398475647), ('lady', 0.7654938697814941), ('boy', 0.7507497072219849), ('girl', 0.7469621896743774), ('soldier', 0.7337659597396851), ('gentleman', 0.7119193077087402), ('kid', 0.682303786277771), ('surgeon', 0.6811317205429077), ('friend', 0.6660101413726807)]


#### 공들여 만든 모델 저장 및 로드하기

In [27]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") #모델 로드

In [28]:
model_result = loaded_model.most_similar("man")
print(model_result) #똑같!!!

[('woman', 0.8388179540634155), ('guy', 0.8041568398475647), ('lady', 0.7654938697814941), ('boy', 0.7507497072219849), ('girl', 0.7469621896743774), ('soldier', 0.7337659597396851), ('gentleman', 0.7119193077087402), ('kid', 0.682303786277771), ('surgeon', 0.6811317205429077), ('friend', 0.6660101413726807)]
